In [ ]:
from scipy.ndimage.filters import convolve
from scipy.signal import convolve2d
from scipy import misc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='white')

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

import tensorflow.keras.backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Dense, Conv2D, MaxPool2D, LSTM,
                                     AvgPool2D, Activation, Flatten,
                                     BatchNormalization, Dropout)
from tensorflow.keras.losses import (mean_squared_error, binary_crossentropy,
                                     categorical_crossentropy)
from tensorflow.keras.optimizers import SGD, Adam, Adagrad, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.metrics import (r2_score, accuracy_score,
                             precision_score, recall_score,
                             f1_score, confusion_matrix,
                             classification_report)
from sklearn.model_selection import (KFold, train_test_split,
                                     cross_val_score)
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.datasets import (make_regression, make_blobs,
                              make_moons, load_iris, load_digits)

%matplotlib inline

# Function

In [ ]:
def plot_decision_boundary(model, X, y, cmap='bwr'):
    amin, bmin = X.min(axis=0) - 0.1
    amax, bmax = X.max(axis=0) + 0.1
    hticks = np.linspace(amin, amax, 101)
    vticks = np.linspace(bmin, bmax, 101)
    
    aa, bb = np.meshgrid(hticks, vticks)
    ab = np.c_[aa.ravel(), bb.ravel()]
    
    c = model.predict(ab)
    cc = c.reshape(aa.shape)
    
    plt.contourf(aa, bb, cc, cmap=cmap, alpha=0.2)
    plt.plot(X[y==0, 0], X[y==0, 1], 'or', alpha=0.5)
    plt.plot(X[y==1, 0], X[y==1, 1], 'ob', alpha=0.5)

# Linear Regression

In [ ]:
K.clear_session()
model = Sequential()
model.add(Dense(1, input_shape=(1,)))
model.compile(loss=mean_squared_error, optimizer=Adam(lr=0.8))
model.summary()

In [ ]:
X, y = make_regression(n_samples=1000, n_features=1,
                       noise=20, random_state=0)
plt.scatter(X, y, color='blue')

In [ ]:
model.fit(X, y, epochs=40, verbose=0)

In [ ]:
yp = model.predict(X)

In [ ]:
plt.scatter(X, y, color='blue')
plt.plot(X, yp, color='red')

In [ ]:
W, b = model.get_weights()
pd.DataFrame([W[0][0], b[0], r2_score(y, yp)],
             index=['W', 'b', 'r2_score'],
             columns=['value'])

# Logistic Regression

In [ ]:
K.clear_session()
model = Sequential()
model.add(Dense(1, input_shape=(1,), activation='sigmoid'))
model.compile(loss=binary_crossentropy, optimizer=SGD(lr=0.5), metrics=['accuracy'])
model.summary()

In [ ]:
X, y = make_blobs(n_samples=1000, n_features=1,
                  centers=2, cluster_std=0.75, random_state=0)
plt.scatter(X, y, color='blue')

In [ ]:
model.fit(X, y, epochs=25, verbose=0)

In [ ]:
yp = model.predict(X) > 0.5

In [ ]:
temp = np.linspace(X.min(axis=0), X.max(axis=0))
plt.scatter(X, y, color='blue')
plt.plot(temp, model.predict(temp), color='red')

In [ ]:
temp = np.linspace(X.min(axis=0), X.max(axis=0))
plt.scatter(X, y, color='blue')
plt.plot(temp, model.predict(temp) > 0.5, color='red')

In [ ]:
W, b = model.get_weights()
pd.DataFrame([W[0][0], b[0], accuracy_score(y, yp)],
             index=['W', 'b', 'accuracy_score'],
             columns=['value'])

# Cross validation

In [ ]:
def build_lr_model():
    model = Sequential()
    model.add(Dense(1, input_shape=(1,), activation='sigmoid'))
    model.compile(loss=binary_crossentropy, optimizer=SGD(lr=0.5), metrics=['accuracy'])
    return model

In [ ]:
model = KerasClassifier(build_fn=build_lr_model, epochs=25, verbose=0)
cv = KFold(3, shuffle=True)
scores = cross_val_score(model, X, y, cv=cv)

In [ ]:
pd.DataFrame([scores.mean(), scores.std()],
             index=['mean', 'std'],
             columns=['cross_val_score'])

# Confusion Matrix

In [ ]:
pd.DataFrame(confusion_matrix(y, yp),
             index=['true:yes', 'true:no'],
             columns=['pred:yes', 'pred:no'])

In [ ]:
pd.DataFrame([precision_score(y, yp), recall_score(y, yp), f1_score(y, yp)],
             index=['precision', 'recall', 'f1'],
             columns=['value'])

In [ ]:
report = classification_report(y, yp, output_dict=True)
pd.DataFrame(report).transpose()

# Feature Preprocessing

In [ ]:
np.random.seed(0)
data = {'X': np.concatenate((np.random.normal(0, 0.2, 500), np.random.normal(1, 0.2, 500))),
        'y': np.array(['a'] * 500 + ['b'] * 500)}
df = pd.DataFrame(data)

In [ ]:
pd.get_dummies(df['y'], prefix='y').head()

In [ ]:
mms = MinMaxScaler()
ss = StandardScaler()
df['X_mms'] = mms.fit_transform(df[['X']])
df['X_ss'] = mms.fit_transform(df[['X']])

In [ ]:
df['X'].hist()
df['X_mms'].hist()
df['X_ss'].hist()

# Machine Learning

In [ ]:
K.clear_session()
model = Sequential()
model.add(Dense(1, input_shape=(2,), activation='sigmoid'))
model.compile(loss=binary_crossentropy, optimizer=Adam(lr=0.05), metrics=['accuracy'])
model.summary()

In [ ]:
X, y = make_moons(n_samples=1000, noise=0.1, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
plt.plot(X[y==0, 0], X[y==0, 1], 'or', alpha=0.5)
plt.plot(X[y==1, 0], X[y==1, 1], 'ob', alpha=0.5)

In [ ]:
model.fit(X_train, y_train, epochs=200, verbose=0)

In [ ]:
res = model.evaluate(X_test, y_test)

In [ ]:
pd.DataFrame(res,
             index=['binary_crossentropy', 'accuracy'],
             columns=['value'])

In [ ]:
plot_decision_boundary(model, X, y)

# Deep Learning

In [ ]:
K.clear_session()
model = Sequential()
model.add(Dense(4, input_shape=(2,), activation='tanh'))
model.add(Dense(2, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss=binary_crossentropy, optimizer=Adam(lr=0.05), metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=200, verbose=0)

In [ ]:
res = model.evaluate(X_test, y_test)

In [ ]:
pd.DataFrame(res,
             index=['binary_crossentropy', 'accuracy'],
             columns=['value'])

In [ ]:
plot_decision_boundary(model, X, y)

# Multi Class

In [ ]:
K.clear_session()
model = Sequential()
model.add(Dense(3, input_shape=(4,), activation='softmax'))
model.compile(loss=categorical_crossentropy, optimizer=Adam(lr=0.1), metrics=['accuracy'])
model.summary()

In [ ]:
data = load_iris()
X = data['data']
y = data['target']
yc = pd.get_dummies(y)
X_train, X_test, y_train, y_test = train_test_split(X, yc, test_size=0.3, random_state=0)

In [ ]:
model.fit(X_train, y_train, epochs=20, verbose=0)

In [ ]:
yp = model.predict(X_test)

In [ ]:
ytc = np.argmax(y_test.values, axis=1)
ypc = np.argmax(yp, axis=1)

In [ ]:
report = classification_report(ytc, ypc, output_dict=True)
pd.DataFrame(report).transpose()

In [ ]:
pd.DataFrame(confusion_matrix(ytc, ypc),
             index=['true:class1', 'true:class2', 'true:class3'],
             columns=['pred:class1', 'pred:class2', 'pred:class3'])

# Activation Function

In [ ]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def step(x):
    return x > 0

def tanh(x):
    return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))

X = np.linspace(-10, 10)
plt.plot(X, sigmoid(X), color='blue')
plt.plot(X, step(X), color='red')
plt.plot(X, tanh(X), color='green')

In [ ]:
def relu(x):
    return x * (x > 0)

def softplus(x):
    return np.log(1 + np.exp(x))

X = np.linspace(-5, 5)
plt.plot(X, relu(X), color='navy')
plt.plot(X, softplus(X), color='purple')

# Learning Rate

In [ ]:
dflist = []
lr = [0.01, 0.05, 0.1, 0.5]
for i in lr:
    K.clear_session()
    model = Sequential()
    model.add(Dense(3, input_shape=(4,), activation='softmax'))
    model.compile(loss=categorical_crossentropy, optimizer=SGD(lr=i), metrics=['accuracy'])
    hist = model.fit(X_train, y_train, batch_size=16, epochs=20, verbose=0)
    dflist.append(pd.DataFrame(hist.history, index=hist.epoch))

histdf = pd.concat(dflist, axis=1)
idx = pd.MultiIndex.from_product([lr, dflist[0].columns], names=['learning_rate','metric'])
histdf.columns = idx

histdf.xs('accuracy', axis=1, level='metric').plot(ax=plt.subplot(211))
histdf.xs('loss', axis=1, level='metric').plot(ax=plt.subplot(212))
plt.tight_layout()

# Batch Size

In [ ]:
dflist = []
bs = [16, 32, 64, 128]
for i in bs:
    K.clear_session()
    model = Sequential()
    model.add(Dense(3, input_shape=(4,), activation='softmax'))
    model.compile(loss=categorical_crossentropy, optimizer=SGD(lr=0.01), metrics=['accuracy'])
    hist = model.fit(X_train, y_train, batch_size=i, epochs=20, verbose=0)
    dflist.append(pd.DataFrame(hist.history, index=hist.epoch))

histdf = pd.concat(dflist, axis=1)
idx = pd.MultiIndex.from_product([bs, dflist[0].columns], names=['batch_size','metric'])
histdf.columns = idx

histdf.xs('accuracy', axis=1, level='metric').plot(ax=plt.subplot(211))
histdf.xs('loss', axis=1, level='metric').plot(ax=plt.subplot(212))
plt.tight_layout()

# Optimizers

In [ ]:
dflist = []
op = ['SGD(lr=0.01)', 'SGD(lr=0.01, momentum=0.3)',
      'SGD(lr=0.01, momentum=0.3, nesterov=True)',
      'Adam(lr=0.01)', 'Adagrad(lr=0.01)', 'RMSprop(lr=0.01)']
for i in op:
    K.clear_session()
    model = Sequential()
    model.add(Dense(3, input_shape=(4,), activation='softmax'))
    model.compile(loss=categorical_crossentropy, optimizer=eval(i), metrics=['accuracy'])
    hist = model.fit(X_train, y_train, batch_size=16, epochs=20, verbose=0)
    dflist.append(pd.DataFrame(hist.history, index=hist.epoch))

histdf = pd.concat(dflist, axis=1)
idx = pd.MultiIndex.from_product([list(range(len(op))), dflist[0].columns], names=['optimizer','metric'])
histdf.columns = idx

histdf.xs('accuracy', axis=1, level='metric').plot(ax=plt.subplot(211))
histdf.xs('loss', axis=1, level='metric').plot(ax=plt.subplot(212))
plt.tight_layout()

# Initialization

In [ ]:
dflist = []
ini = ['zeros', 'uniform', 'normal', 'he_normal', 'lecun_uniform']
for i in ini:
    K.clear_session()
    model = Sequential()
    model.add(Dense(3, input_shape=(4,), kernel_initializer=i, activation='softmax'))
    model.compile(loss=categorical_crossentropy, optimizer=RMSprop(), metrics=['accuracy'])
    hist = model.fit(X_train, y_train, batch_size=16, epochs=20, verbose=0)
    dflist.append(pd.DataFrame(hist.history, index=hist.epoch))

histdf = pd.concat(dflist, axis=1)
idx = pd.MultiIndex.from_product([ini, dflist[0].columns], names=['optimizer','metric'])
histdf.columns = idx

histdf.xs('accuracy', axis=1, level='metric').plot(ax=plt.subplot(211))
histdf.xs('loss', axis=1, level='metric').plot(ax=plt.subplot(212))
plt.tight_layout()

# Visualization

In [ ]:
K.clear_session()
model = Sequential()
model.add(Dense(3, input_shape=(4,), activation='softmax'))
model.compile(loss=categorical_crossentropy, optimizer=Adam(lr=0.1), metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=20, verbose=0)

In [ ]:
ff = K.function([model.layers[0].input], [model.layers[0].output])
f = ff([X_test])[0]
plt.scatter(f[:, 0], f[:, 1], c=np.argmax(y_test.values, axis=1))

# Layer Visualization

In [ ]:
K.clear_session()
model = Sequential()
model.add(Dense(3, input_shape=(2,), activation='softmax'))
model.compile(loss=categorical_crossentropy, optimizer=Adam(lr=0.1), metrics=['accuracy'])
model.summary()

In [ ]:
X, y = make_blobs(n_samples=1000, n_features=2,
                  centers=3, cluster_std=0.5, random_state=0)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='tab20b')

In [ ]:
plt.figure(figsize=(15,10))
for i in range(1, 26):
    model.fit(X, pd.get_dummies(y), epochs=1, verbose=0)
    ff = K.function([model.layers[0].input], [model.layers[0].output])
    f = ff([X])[0]
    plt.subplot(5, 5, i)
    plt.scatter(f[:, 0], f[:, 1], c=y, cmap='tab20b')
plt.tight_layout()

# MNIST

In [ ]:
K.clear_session()
model = Sequential()
model.add(Dense(512, input_dim=28*28, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss=categorical_crossentropy, optimizer=RMSprop(), metrics=['accuracy'])
model.summary()

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
plt.imshow(X_train[0], cmap='gray')

In [ ]:
X_train = X_train.reshape(-1, 28*28) / 255.0
X_test = X_test.reshape(-1, 28*28) / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
hist = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_split=0.3, verbose=0)
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])

In [ ]:
res = model.evaluate(X_test, y_test)

In [ ]:
pd.DataFrame(res,
             index=['categorical_crossentropy', 'accuracy'],
             columns=['value'])

# Convolution

In [ ]:
a = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0])
b = np.array([-1, 1])
c = np.convolve(a, b)
plt.subplot(211)
plt.plot(a, 'o-')
plt.subplot(212)
plt.plot(c, 'o-')

In [ ]:
img = misc.ascent()
plt.imshow(img, cmap='gray')

In [ ]:
h_kernel = np.array([[ 1,  2,  1],
                  [ 0,  0,  0],
                  [-1, -2, -1]])
plt.imshow(h_kernel, cmap='gray')

In [ ]:
res = convolve2d(img, h_kernel)
plt.imshow(res, cmap='gray')

# Conv Layer

In [ ]:
K.clear_session()
model = Sequential()
model.add(Conv2D(1, (3, 3), strides=(2, 1), input_shape=(512, 512, 1)))
model.compile(loss=mean_squared_error, optimizer=Adam())
model.summary()

In [ ]:
img_tensor = img.reshape((1, 512, 512, 1))
img_tensor_p = model.predict(img_tensor)
plt.imshow(img_tensor_p[0, :, :, 0], cmap='gray')

In [ ]:
weights = model.get_weights()
plt.imshow(weights[0][:, :, 0, 0], cmap='gray')

In [ ]:
model.set_weights([np.ones(w.shape) for w in weights])
img_tensor_p = model.predict(img_tensor)
plt.imshow(img_tensor_p[0, :, :, 0], cmap='gray')

In [ ]:
K.clear_session()
model = Sequential()
model.add(Conv2D(1, (3, 3), padding='same', input_shape=(512, 512, 1)))
model.compile(loss=mean_squared_error, optimizer=Adam())
model.summary()

In [ ]:
img_tensor = img.reshape((1, 512, 512, 1))
img_tensor_p = model.predict(img_tensor)
plt.imshow(img_tensor_p[0, :, :, 0], cmap='gray')

# Pool Layer

In [ ]:
K.clear_session()
model = Sequential()
model.add(MaxPool2D(1, (5, 5), input_shape=(512, 512, 1)))
model.compile(loss=mean_squared_error, optimizer=Adam())
model.summary()

In [ ]:
img_tensor = img.reshape((1, 512, 512, 1))
img_tensor_p = model.predict(img_tensor)
plt.imshow(img_tensor_p[0, :, :, 0], cmap='gray')

In [ ]:
K.clear_session()
model = Sequential()
model.add(AvgPool2D(1, (5, 5), input_shape=(512, 512, 1)))
model.compile(loss=mean_squared_error, optimizer=Adam())
model.summary()

In [ ]:
img_tensor = img.reshape((1, 512, 512, 1))
img_tensor_p = model.predict(img_tensor)
plt.imshow(img_tensor_p[0, :, :, 0], cmap='gray')

# Convolutional Neural Network

In [ ]:
K.clear_session()
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss=categorical_crossentropy, optimizer=RMSprop(), metrics=['accuracy'])
model.summary()

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
plt.imshow(X_train[0], cmap='gray')

In [ ]:
X_train = X_train.reshape(-1, 28, 28, 1) / 255.0
X_test = X_test.reshape(-1, 28, 28, 1) / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
hist = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_split=0.3, verbose=0)
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])

In [ ]:
res = model.evaluate(X_test, y_test)

In [ ]:
pd.DataFrame(res,
             index=['categorical_crossentropy', 'accuracy'],
             columns=['value'])

# Forecast

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv', header=0, index_col=0, names=['value'])
plt.plot(df, color='blue')

In [ ]:
cut = round(df.shape[0] * 0.7)
X_train = df.iloc[:cut].copy()
X_test = df.iloc[cut:].copy()
plt.plot(X_train, color='blue')
plt.plot(X_test, color='red')

In [ ]:
mms = MinMaxScaler()
X_train['value'] = mms.fit_transform(X_train)
X_test['value'] = mms.transform(X_test)
plt.plot(X_train, color='blue')
plt.plot(X_test, color='red')

In [ ]:
def ts_shift(df, lag=20):
    df = df.copy()
    for i in range(1, lag + 1):
        df['lag{:02d}'.format(i)] = df['value'].shift(i)
    df.dropna(axis=0, inplace=True)
    return df

X_train_s = ts_shift(X_train)
X_test_s = ts_shift(X_test)

X_train = X_train_s.drop('value', axis=1).values
y_train = X_train_s[['value']].values

X_test = X_test_s.drop('value', axis=1).values
y_test = X_test_s[['value']].values

In [ ]:
K.clear_session()
model = Sequential()
model.add(Dense(20, input_dim=20, activation='relu'))
model.add(Dense(1))
model.compile(loss=mean_squared_error, optimizer=Adam(), metrics=['accuracy'])
model.summary()

In [ ]:
early_stop = EarlyStopping(monitor='loss', patience=1, verbose=1)
hist = model.fit(X_train, y_train, batch_size=2, epochs=200, validation_split=0.3, verbose=0, callbacks=[early_stop])
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])

In [ ]:
yp = model.predict(X_test)

In [ ]:
plt.plot(y_test, color='blue')
plt.plot(yp, color='red')

In [ ]:
res = model.evaluate(X_test, y_test)

In [ ]:
pd.DataFrame(res + [r2_score(y_test, yp)],
             index=['mean_squared_error', 'accuracy', 'r2_score'],
             columns=['value'])

# LSTM

In [ ]:
K.clear_session()
model = Sequential()
model.add(LSTM(10, input_shape=(1, 20)))
model.add(Dense(1))
model.compile(loss=mean_squared_error, optimizer=Adam(), metrics=['accuracy'])
model.summary()

In [ ]:
X_train_aux = X_train[:, None]
X_test_aux = X_test[:, None]

In [ ]:
early_stop = EarlyStopping(monitor='loss', patience=1, verbose=0)
hist = model.fit(X_train_aux, y_train, batch_size=2, epochs=200, validation_split=0.3, verbose=0, callbacks=[early_stop])
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])

In [ ]:
yp = model.predict(X_test_aux)

In [ ]:
plt.plot(y_test, color='blue')
plt.plot(yp, color='red')

In [ ]:
res = model.evaluate(X_test_aux, y_test)

In [ ]:
pd.DataFrame(res + [r2_score(y_test, yp)],
             index=['mean_squared_error', 'accuracy', 'r2_score'],
             columns=['value'])

# Learning Curves

In [ ]:
K.clear_session()
model = Sequential()
model.add(Dense(16, input_dim=64, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
model.summary()

In [ ]:
data = load_digits()
X, y = data.data, data.target
yc = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X, yc, test_size=0.3, random_state=0)
plt.imshow(X.reshape(-1, 8, 8)[0], cmap='gray')

In [ ]:
initial_weights = model.get_weights()

In [ ]:
train_size = (len(X_train) * np.linspace(0.1, 0.999, 4)).astype(int)

In [ ]:
train_score = []
test_score = []

for ts in train_size:
    X_train_frac, _, y_train_frac, _ = train_test_split(X_train, y_train, train_size=ts)
    
    model.set_weights(initial_weights)
    
    early_stop = EarlyStopping(monitor='loss', patience=1, verbose=0)
    hist = model.fit(X_train_frac, y_train_frac, epochs=300, validation_split=0.3, verbose=0, callbacks=[early_stop])
    
    aux = model.evaluate(X_train_frac, y_train_frac, verbose=0)
    train_score.append(aux[-1])
    
    aux = model.evaluate(X_test, y_test, verbose=0)
    test_score.append(aux[-1])

plt.plot(train_size, train_score, color='blue')
plt.plot(train_size, test_score, color='red')

# Batch Normalization

In [ ]:
def repeat_train(X_train, y_train, X_test, y_test, repeat=3, bn=False):
    ll = []
    
    for r in range(repeat):
        K.clear_session()
        model = Sequential()

        model.add(Dense(512, input_shape=X_train.shape[1:], kernel_initializer='normal', activation='sigmoid'))

        if bn:
            model.add(BatchNormalization())

        model.add(Dense(512, kernel_initializer='normal', activation='sigmoid'))

        if bn:
            model.add(BatchNormalization())

        model.add(Dense(512, kernel_initializer='normal', activation='sigmoid'))

        if bn:
            model.add(BatchNormalization())

        model.add(Dense(10, activation='softmax'))

        model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
        hist = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, verbose=0)

        ll.append([hist.history['accuracy'], hist.history['val_accuracy']])
    
    ll = np.array(ll)
    mean_acc = ll.mean(axis=0)
    std_acc = ll.std(axis=0)
    
    return mean_acc[0], std_acc[0], mean_acc[1], std_acc[1]

mean_acc, std_acc, mean_acc_val, std_acc_val = repeat_train(X_train, y_train, X_test, y_test, bn=False)
mean_acc_bn, std_acc_bn, mean_acc_val_bn, std_acc_val_bn = repeat_train(X_train, y_train, X_test, y_test, bn=True)

In [ ]:
def plot_mean_std(m, s):
    plt.plot(m)
    plt.fill_between(range(len(m)), m-s, m+s, alpha=0.1)
plot_mean_std(mean_acc, std_acc)
plot_mean_std(mean_acc_val, std_acc_val)
plot_mean_std(mean_acc_bn, std_acc_bn)
plot_mean_std(mean_acc_val_bn, std_acc_val_bn)
plt.legend(['train', 'test', 'train_bn', 'test_bn'])

# Regularizer and Dropout

In [ ]:
model = Sequential()
model.add(Dropout(0.2, input_shape=X_train.shape[1:]))
model.add(Dense(512, kernel_initializer='normal', kernel_regularizer='l2', activation='sigmoid'))
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))
model.compile(loss=categorical_crossentropy, optimizer=SGD(), metrics=['accuracy'])
model.summary()

In [ ]:
hist = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_split=0.3, verbose=0)
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])

In [ ]:
res = model.evaluate(X_test, y_test)

In [ ]:
pd.DataFrame(res,
             index=['categorical_crossentropy', 'accuracy'],
             columns=['value'])